In [9]:
import multiprocessing

from tqdm import tqdm
import newspaper
import requests
import ujson

In [4]:
def download_all_results(api_token, crawl_name, out_path):
    with open(out_path, 'w') as _out:
        response = requests.get('https://' + api_token + ':@api.80legs.com/v2/results/' + crawl_name)
        result_files = response.json()
        for result_file in tqdm(result_files): 
            file_response = requests.get(result_file, stream=True)
            file_content = bytearray()
            for chunk in file_response.iter_content(chunk_size=1024):
                file_content.extend(chunk)

            try:
                results = ujson.loads(file_content.decode('utf-8'))
                for result in results: 
                    _out.write(ujson.dumps(result) + '\n')
            except Exception:
                print('Failed :(', crawl_name)

In [6]:
download_all_results('zkznwmuog07v6zjnef1igo32h80jtwo2', 'FakeNews_1', 'data/11_80legs/FakeNews_1_result.json')

100%|██████████| 146/146 [23:49<00:00,  9.79s/it]


In [11]:
def process_result(result):
    result = ujson.loads(result)

    if 'url' not in result or 'result' not in result:
        return

    url = result['url']
    html = result['result']

    if url is None or html is None or len(html) <= 1:
        return

    article = newspaper.Article(url)
    article.set_html(html)
    article.parse()

    return ujson.dumps({
        'title': article.title,
        'body': article.text,
        'meta': {
            'original_id': url,
            'url': url,
            'published_at': article.publish_date,
            'raw': {
                'images': article.images,
                'favicon': article.meta_favicon,
                'tags': article.tags,
                'authors': article.authors,
                'top_image': article.top_image,
                'html': html
            }
        }
    }) + '\n'

In [ ]:
with open('data/11_80legs/FakeNews_1_results_parsed.json', 'w') as _out: 
    with open('data/11_80legs/FakeNews_1_result.json', 'r') as _in: 
        bar = tqdm()
        pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
        for result in pool.imap(process_result, _in, 10):
            if result is None: 
                continue

            _out.write(result)
            bar.update()


0it [00:00, ?it/s]




1it [00:04,  4.00s/it]
41it [00:06,  2.82s/it]
51it [00:06,  2.00s/it]
71it [00:07,  1.40s/it]
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

12471it [11:48, 18.67it/s]WARNING:newspaper.parsers:fromstring() returned an invalid string: <!–...
13981it [12:55, 22.94it/s]WARNING:newspaper.parsers:fromstring() returned an invalid string: <!–...
22211it [24:38,  9.52it/s]/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5818 bytes but only got 50. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/l

/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32768 bytes but only got 18642. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32768 bytes but only got 19666. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32768 bytes but only got 20690. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32768 bytes but only got 21714. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXI